In [ ]:
import os
from glob import glob


In [ ]:
means = [0.485, 0.456, 0.406]
stds = [0.229, 0.224, 0.225]

t_stds = torch.tensor(stds).cuda().half()[:,None,None]
t_means = torch.tensor(means).cuda().half()[:,None,None]

img_transforms = transforms.Compose([                        
            transforms.ToTensor(),
            transforms.Normalize(means,stds)])
 
def tensor2im(var):
     return var.mul(t_stds).add(t_means).mul(255.).clamp(0,255).permute(1,2,0)

def proc_pil_img(input_image, model):
    transformed_image = img_transforms(input_image)[None,...].cuda().half()
            
    with torch.no_grad():
        result_image = model(transformed_image)[0]; print(result_image.shape)
        output_image = tensor2im(result_image)
        output_image = output_image.detach().cpu().numpy().astype('uint8')
        output_image = PIL.Image.fromarray(output_image)
    return output_image


In [ ]:
def process(src_path, dest_path):
  in_files = sorted(glob(f'{src_path}/*'))
  for img in tqdm(in_files):
    out = f"{dest_path}/{img.split('/')[-1].split('.')[0]}.jpg"
    im = PIL.Image.open(img).resize((512, 512)).convert("RGB") 
    im = scale_by_face_size(im, target_face=300, max_res=1_500_000, max_upscale=1)
    res = proc_pil_img(im, model)
    res.save(out)


## 텐서를 이미지로

In [7]:
import torch
import numpy as np
from PIL import Image
def tensor2im(x) -> Image.Image:
    # x: [C, H, W] or [H, W]
    x = x.detach().cpu()
    
    if x.dim() == 3:
        # [C, H, W] → [H, W, C]
        x = x.permute(1, 2, 0)

    x = x.numpy()
    x = (x * 255).clip(0, 255).astype(np.uint8)

    return Image.fromarray(x)


x = torch.rand(3, 256, 256) # 값 범위: [0, 1] (float형)
image = tensor2im(x)
image.save("/home/torch_pipeline/data_preparation/vision/output.png")


In [ ]:
# PIL로 바꿔서 저장
import torch
from torchvision.transforms import ToPILImage

tensor = torch.randn(3, 256, 256)  # 예시: 값은 [-1, 1]일 수 있음
# 값이 [0, 1] 범위로 들어가야 PIL 변환 가능하므로 clamp 후 normalize
tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min())

# 텐서를 PIL 이미지로 변환
to_pil = ToPILImage()
image = to_pil(tensor)

# 저장
image.save("output.png")

In [ ]:
# PIL 이미지를 그리드로 저장
def image_grid(imgs:PIL.Image, rows:int, cols:int):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

## 이미지 다운

In [ ]:
import requests
from io import BytesIO
from PIL import Image

def download_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content)).convert("RGB")

ip_adapter_image = download_image("https://cdn-uploads.huggingface.co/production/uploads/1668693456211-noauth.jpeg")


### bbox 그리기

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import os

# 경로
image_path = "/purestorage/AILAB/AI_1/dataset/Objects365/images/train/objects365_v1_00000000.jpg"
label_path = "/purestorage/AILAB/AI_1/dataset/Objects365/labels/train/objects365_v1_00000000.txt"

# 이미지 로드
image = Image.open(image_path).convert("RGB")
width, height = image.size
draw = ImageDraw.Draw(image)

# 라벨 파일 열기
with open(label_path, "r") as f:
    lines = f.readlines()

for line in lines:
    parts = line.strip().split()
    if len(parts) != 5:
        continue

    class_id, x_center, y_center, box_width, box_height = map(float, parts)

    # 정규화 좌표 → 픽셀 좌표로 변환
    x_center *= width
    y_center *= height
    box_width *= width
    box_height *= height

    x0 = max(x_center - box_width / 2, 0)
    y0 = max(y_center - box_height / 2, 0)
    x1 = min(x_center + box_width / 2, width)
    y1 = min(y_center + box_height / 2, height)

    # 박스 그리기 (굵기 강조)
    # 20.29876000000001 260.25216 102.9964 305.77408
    draw.rectangle([x0, y0, x1, y1], outline="red", width=3)

    # class_id도 텍스트로 표시
    draw.text((x0 + 2, y0 + 2), str(int(class_id)), fill="yellow")

### 그리드 만들어서 저장

import os
import torch
import torchvision.transforms as transforms
import torchvision.utils as vutils
from PIL import Image

def resize_image(image):
    return image.resize((256, 256))

def create_image_grid(image_folder):
    transform = transforms.ToTensor()

    # 이미지를 읽고 텐서로 변환하여 리스트에 저장
    grid_images = []
    for i in range(0, 360, 10):
        # 랜드마크 이미지
        kps_image_path = os.path.join(image_folder, f'kps_{i}.png')
        kps_image = Image.open(kps_image_path).convert('RGB').resize((256, 256))
        grid_images.append(transform(kps_image))

        # 생성된 얼굴 이미지
        out_image_path = os.path.join(image_folder, f'out_{i}.png')
        out_image = Image.open(out_image_path).convert('RGB').resize((256, 256))
        grid_images.append(transform(out_image))

    # 이미지들을 그리드로 만들기
    grid = vutils.make_grid(grid_images, nrow=2, padding=2, normalize=True)

    # 그리드 이미지 저장 또는 시각화
    vutils.save_image(grid, 'combined_grid.jpg')

    return grid

# 이미지 개수와 이미지 폴더
num_images = 10

image_folder = '/purestorage/project/tyk/2_ImageGenProjects/InstantID'  # 이미지가 저장된 폴더 경로
grid = create_image_grid(image_folder)